# DBSCAN - Complete Guide
## Density-Based Spatial Clustering of Applications with Noise

## 📚 Learning Objectives
- Understand DBSCAN algorithm and density-based clustering
- Learn to tune eps and min_samples parameters
- Detect and handle noise/outliers
- Find clusters of arbitrary shapes
- Compare DBSCAN with K-Means
- Apply to real-world geospatial and anomaly detection problems

## 🎯 What is DBSCAN?

**DBSCAN** is a density-based clustering algorithm that groups together points that are closely packed together, marking points in low-density regions as outliers.

### Key Concepts:

1. **Core Point**: A point with at least `min_samples` neighbors within `eps` distance
2. **Border Point**: A point within `eps` distance of a core point, but has fewer than `min_samples` neighbors
3. **Noise Point**: A point that is neither core nor border (outlier)

### Parameters:
- **eps (ε)**: Maximum distance between two points to be considered neighbors
- **min_samples**: Minimum number of points required to form a dense region (core point)

### Advantages over K-Means:
✅ **No need to specify number of clusters** (K-Means requires K)  
✅ **Finds arbitrary-shaped clusters** (K-Means only finds spherical)  
✅ **Detects outliers/noise** (K-Means assigns all points to clusters)  
✅ **Robust to outliers** (K-Means sensitive to outliers)  

### When to Use:
✅ Unknown number of clusters  
✅ Non-spherical cluster shapes  
✅ Presence of noise/outliers  
✅ Geospatial data  
✅ Anomaly detection  

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_blobs
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

## Part 1: Understanding DBSCAN with Synthetic Data
### 1️⃣ Create Non-Spherical Clusters

In [ ]:
# Create three different datasets with different shapes
np.random.seed(42)

# Dataset 1: Moons (non-linear, crescent shapes)
X_moons, y_moons = make_moons(n_samples=300, noise=0.05, random_state=42)

# Dataset 2: Circles (concentric circles)
X_circles, y_circles = make_circles(n_samples=300, noise=0.05, factor=0.5, random_state=42)

# Dataset 3: Blobs with noise (spherical + outliers)
X_blobs, y_blobs = make_blobs(n_samples=300, centers=3, cluster_std=0.5, random_state=42)
# Add noise points
noise = np.random.uniform(X_blobs.min(), X_blobs.max(), size=(30, 2))
X_blobs = np.vstack([X_blobs, noise])
y_blobs = np.hstack([y_blobs, -1 * np.ones(30)])  # -1 for noise

# Visualize datasets
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets = [
    (X_moons, 'Moons (Crescent Shapes)'),
    (X_circles, 'Circles (Concentric)'),
    (X_blobs, 'Blobs with Noise')
]

for idx, (X, title) in enumerate(datasets):
    axes[idx].scatter(X[:, 0], X[:, 1], c='steelblue', alpha=0.6, edgecolors='black', s=50)
    axes[idx].set_title(title, fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Feature 1', fontsize=11)
    axes[idx].set_ylabel('Feature 2', fontsize=11)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 These datasets have non-spherical shapes that K-Means struggles with")

### 2️⃣ Compare K-Means vs DBSCAN

In [ ]:
# Apply K-Means and DBSCAN to moons dataset
X = X_moons

# K-Means
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X)

# DBSCAN
dbscan = DBSCAN(eps=0.3, min_samples=5)
dbscan_labels = dbscan.fit_predict(X)

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Original data
axes[0].scatter(X[:, 0], X[:, 1], c='gray', alpha=0.6, edgecolors='black', s=50)
axes[0].set_title('Original Data', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1', fontsize=11)
axes[0].set_ylabel('Feature 2', fontsize=11)
axes[0].grid(True, alpha=0.3)

# K-Means clustering
scatter1 = axes[1].scatter(X[:, 0], X[:, 1], c=kmeans_labels, cmap='viridis', 
                           alpha=0.6, edgecolors='black', s=50)
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
               c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centroids')
axes[1].set_title('K-Means Clustering (K=2)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1', fontsize=11)
axes[1].set_ylabel('Feature 2', fontsize=11)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# DBSCAN clustering
scatter2 = axes[2].scatter(X[:, 0], X[:, 1], c=dbscan_labels, cmap='viridis',
                           alpha=0.6, edgecolors='black', s=50)
# Highlight noise points
noise_mask = dbscan_labels == -1
if noise_mask.any():
    axes[2].scatter(X[noise_mask, 0], X[noise_mask, 1], c='red', marker='x',
                   s=100, linewidths=2, label='Noise')
axes[2].set_title(f'DBSCAN (eps=0.3, min_samples=5)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Feature 1', fontsize=11)
axes[2].set_ylabel('Feature 2', fontsize=11)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print(f"\n📊 Clustering Results:")
print(f"\nK-Means:")
print(f"  - Clusters found: {len(set(kmeans_labels))}")
print(f"  - Silhouette Score: {silhouette_score(X, kmeans_labels):.4f}")

print(f"\nDBSCAN:")
print(f"  - Clusters found: {n_clusters_dbscan}")
print(f"  - Noise points: {n_noise}")
if n_clusters_dbscan > 1:
    # Calculate silhouette score only for non-noise points
    mask = dbscan_labels != -1
    if mask.sum() > 0:
        print(f"  - Silhouette Score: {silhouette_score(X[mask], dbscan_labels[mask]):.4f}")

print("\n💡 DBSCAN correctly identifies the crescent shapes, while K-Means fails!")

### 3️⃣ Finding Optimal eps Parameter
#### Using K-distance Graph (Elbow Method for DBSCAN)

In [ ]:
# Calculate k-distance for each point
# k should be min_samples - 1
min_samples = 5
k = min_samples - 1

# Fit nearest neighbors
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(X)
distances, indices = neighbors_fit.kneighbors(X)

# Sort distances
distances = np.sort(distances[:, k-1], axis=0)

# Plot k-distance graph
plt.figure(figsize=(12, 6))
plt.plot(distances, linewidth=2, color='steelblue')
plt.xlabel('Points sorted by distance', fontsize=12)
plt.ylabel(f'{k}-NN Distance', fontsize=12)
plt.title(f'K-Distance Graph (k={k})\nLook for "elbow" to determine optimal eps', 
         fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Add suggested eps line
suggested_eps = 0.3
plt.axhline(y=suggested_eps, color='red', linestyle='--', linewidth=2, 
           label=f'Suggested eps = {suggested_eps}')
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("\n💡 The 'elbow' in the graph suggests a good eps value")
print("💡 Points after the elbow are likely outliers")

### 4️⃣ Parameter Tuning: eps and min_samples

In [ ]:
# Test different parameter combinations
eps_values = [0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_values = [3, 5, 10, 15]

results = []

for eps in eps_values:
    for min_samp in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samp)
        labels = dbscan.fit_predict(X)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        # Calculate silhouette score if we have clusters
        if n_clusters > 1:
            mask = labels != -1
            if mask.sum() > 0 and len(set(labels[mask])) > 1:
                sil_score = silhouette_score(X[mask], labels[mask])
            else:
                sil_score = -1
        else:
            sil_score = -1
        
        results.append({
            'eps': eps,
            'min_samples': min_samp,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'silhouette': sil_score
        })

results_df = pd.DataFrame(results)
print("\n📊 Parameter Tuning Results:")
print(results_df.to_string(index=False))

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap of number of clusters
pivot_clusters = results_df.pivot(index='min_samples', columns='eps', values='n_clusters')
sns.heatmap(pivot_clusters, annot=True, fmt='g', cmap='YlOrRd', ax=axes[0], 
           cbar_kws={'label': 'Number of Clusters'})
axes[0].set_title('Number of Clusters Found', fontsize=14, fontweight='bold')
axes[0].set_xlabel('eps', fontsize=12)
axes[0].set_ylabel('min_samples', fontsize=12)

# Heatmap of silhouette scores
pivot_sil = results_df.pivot(index='min_samples', columns='eps', values='silhouette')
sns.heatmap(pivot_sil, annot=True, fmt='.3f', cmap='RdYlGn', ax=axes[1],
           cbar_kws={'label': 'Silhouette Score'})
axes[1].set_title('Silhouette Scores', fontsize=14, fontweight='bold')
axes[1].set_xlabel('eps', fontsize=12)
axes[1].set_ylabel('min_samples', fontsize=12)

plt.tight_layout()
plt.show()

# Find best parameters
best_params = results_df.loc[results_df['silhouette'].idxmax()]
print(f"\n🎯 Best Parameters:")
print(f"eps = {best_params['eps']}")
print(f"min_samples = {int(best_params['min_samples'])}")
print(f"Silhouette Score = {best_params['silhouette']:.4f}")

## Part 2: Real-World Application - Customer Segmentation
### 5️⃣ Load and Prepare Data

In [ ]:
# Create synthetic customer data
np.random.seed(42)

# Generate customer segments
n_customers = 500

# Segment 1: High value customers (high spending, high frequency)
high_value = np.random.multivariate_normal(
    mean=[80, 15], 
    cov=[[100, 20], [20, 10]], 
    size=100
)

# Segment 2: Regular customers (medium spending, medium frequency)
regular = np.random.multivariate_normal(
    mean=[50, 8],
    cov=[[80, 15], [15, 8]],
    size=200
)

# Segment 3: Occasional customers (low spending, low frequency)
occasional = np.random.multivariate_normal(
    mean=[20, 3],
    cov=[[50, 10], [10, 5]],
    size=150
)

# Add some outliers (one-time big purchases, fraudulent transactions, etc.)
outliers = np.random.uniform(low=[0, 0], high=[100, 20], size=(50, 2))

# Combine all data
X_customers = np.vstack([high_value, regular, occasional, outliers])

# Create DataFrame
df_customers = pd.DataFrame(X_customers, columns=['Total_Spending', 'Purchase_Frequency'])
df_customers['Total_Spending'] = df_customers['Total_Spending'].clip(lower=0)
df_customers['Purchase_Frequency'] = df_customers['Purchase_Frequency'].clip(lower=0)

print(f"Customer dataset shape: {df_customers.shape}")
print(f"\n📊 Dataset Statistics:")
print(df_customers.describe())

# Visualize
plt.figure(figsize=(12, 6))
plt.scatter(df_customers['Total_Spending'], df_customers['Purchase_Frequency'],
           alpha=0.6, edgecolors='black', s=50, c='steelblue')
plt.xlabel('Total Spending ($)', fontsize=12)
plt.ylabel('Purchase Frequency (per month)', fontsize=12)
plt.title('Customer Data Distribution', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 6️⃣ Apply DBSCAN to Customer Data

In [ ]:
# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_customers)

# Apply DBSCAN
dbscan_customers = DBSCAN(eps=0.5, min_samples=10)
clusters = dbscan_customers.fit_predict(X_scaled)

# Add cluster labels to dataframe
df_customers['Cluster'] = clusters

# Statistics
n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
n_noise = list(clusters).count(-1)

print(f"\n📊 DBSCAN Results:")
print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points (outliers): {n_noise}")
print(f"\nCluster sizes:")
print(df_customers['Cluster'].value_counts().sort_index())

# Visualize clusters
plt.figure(figsize=(14, 7))

# Plot each cluster
unique_clusters = set(clusters)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_clusters)))

for cluster, color in zip(unique_clusters, colors):
    if cluster == -1:
        # Noise points in black
        mask = clusters == cluster
        plt.scatter(df_customers.loc[mask, 'Total_Spending'],
                   df_customers.loc[mask, 'Purchase_Frequency'],
                   c='black', marker='x', s=100, linewidths=2,
                   label='Outliers', alpha=0.8)
    else:
        mask = clusters == cluster
        plt.scatter(df_customers.loc[mask, 'Total_Spending'],
                   df_customers.loc[mask, 'Purchase_Frequency'],
                   c=[color], s=80, alpha=0.7, edgecolors='black',
                   label=f'Cluster {cluster}')

plt.xlabel('Total Spending ($)', fontsize=12)
plt.ylabel('Purchase Frequency (per month)', fontsize=12)
plt.title('Customer Segmentation with DBSCAN', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 7️⃣ Cluster Profiling and Business Insights

In [ ]:
# Calculate cluster statistics
cluster_profiles = df_customers.groupby('Cluster').agg({
    'Total_Spending': ['mean', 'median', 'std', 'count'],
    'Purchase_Frequency': ['mean', 'median', 'std']
}).round(2)

print("\n📊 Cluster Profiles:")
print(cluster_profiles)

# Assign business labels
def assign_segment_label(row):
    if row['Cluster'] == -1:
        return 'Outlier/Anomaly'
    elif row['Total_Spending'] > 70 and row['Purchase_Frequency'] > 12:
        return 'VIP Customers'
    elif row['Total_Spending'] > 40 and row['Purchase_Frequency'] > 6:
        return 'Regular Customers'
    elif row['Total_Spending'] < 40:
        return 'Occasional Customers'
    else:
        return 'Other'

df_customers['Segment'] = df_customers.apply(assign_segment_label, axis=1)

print("\n📊 Business Segments:")
print(df_customers['Segment'].value_counts())

# Visualize segments
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Segment distribution
segment_counts = df_customers['Segment'].value_counts()
axes[0].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%',
           startangle=90, colors=sns.color_palette('Set3'))
axes[0].set_title('Customer Segment Distribution', fontsize=14, fontweight='bold')

# Box plots
df_customers.boxplot(column='Total_Spending', by='Segment', ax=axes[1])
axes[1].set_title('Spending Distribution by Segment', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Segment', fontsize=12)
axes[1].set_ylabel('Total Spending ($)', fontsize=12)
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

# Business recommendations
print("\n💼 Business Insights & Recommendations:")
print("\n1. VIP Customers:")
print("   - High value, high frequency")
print("   - Action: Loyalty programs, exclusive offers, personal account manager")
print("\n2. Regular Customers:")
print("   - Medium value, medium frequency")
print("   - Action: Upselling campaigns, rewards program")
print("\n3. Occasional Customers:")
print("   - Low value, low frequency")
print("   - Action: Re-engagement campaigns, special discounts")
print("\n4. Outliers/Anomalies:")
print("   - Unusual patterns")
print("   - Action: Investigate for fraud, one-time events, or data errors")

## Part 3: Geospatial Application
### 8️⃣ Crime Hotspot Detection

In [ ]:
# Generate synthetic crime location data
np.random.seed(42)

# Hotspot 1: Downtown area
downtown = np.random.multivariate_normal(
    mean=[40.7589, -73.9851],  # NYC coordinates
    cov=[[0.0001, 0], [0, 0.0001]],
    size=150
)

# Hotspot 2: Another high-crime area
area2 = np.random.multivariate_normal(
    mean=[40.7489, -73.9651],
    cov=[[0.00008, 0], [0, 0.00008]],
    size=100
)

# Scattered incidents
scattered = np.random.uniform(
    low=[40.72, -74.01],
    high=[40.78, -73.95],
    size=(50, 2)
)

# Combine
crime_locations = np.vstack([downtown, area2, scattered])
df_crime = pd.DataFrame(crime_locations, columns=['Latitude', 'Longitude'])

print(f"Crime incidents: {len(df_crime)}")
print(f"\n📊 Location Statistics:")
print(df_crime.describe())

# Apply DBSCAN (eps in degrees, approximately 100 meters = 0.001 degrees)
dbscan_crime = DBSCAN(eps=0.003, min_samples=10)
crime_clusters = dbscan_crime.fit_predict(df_crime)

df_crime['Hotspot'] = crime_clusters

n_hotspots = len(set(crime_clusters)) - (1 if -1 in crime_clusters else 0)
n_isolated = list(crime_clusters).count(-1)

print(f"\n📊 Crime Hotspot Analysis:")
print(f"Number of hotspots detected: {n_hotspots}")
print(f"Isolated incidents: {n_isolated}")
print(f"\nHotspot sizes:")
print(df_crime['Hotspot'].value_counts().sort_index())

# Visualize
plt.figure(figsize=(14, 10))

unique_hotspots = set(crime_clusters)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_hotspots)))

for hotspot, color in zip(unique_hotspots, colors):
    if hotspot == -1:
        mask = crime_clusters == hotspot
        plt.scatter(df_crime.loc[mask, 'Longitude'],
                   df_crime.loc[mask, 'Latitude'],
                   c='gray', marker='o', s=30, alpha=0.5,
                   label='Isolated Incidents')
    else:
        mask = crime_clusters == hotspot
        plt.scatter(df_crime.loc[mask, 'Longitude'],
                   df_crime.loc[mask, 'Latitude'],
                   c=[color], s=100, alpha=0.7, edgecolors='black',
                   label=f'Hotspot {hotspot} ({mask.sum()} incidents)')

plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)
plt.title('Crime Hotspot Detection using DBSCAN', fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🚔 Law Enforcement Recommendations:")
print("1. Deploy additional patrols to identified hotspots")
print("2. Investigate common factors in hotspot areas")
print("3. Implement preventive measures in high-density zones")
print("4. Monitor isolated incidents for emerging patterns")

## 📊 Key Takeaways

### DBSCAN Advantages:
✅ **No need to specify K** - Automatically determines number of clusters  
✅ **Arbitrary shapes** - Finds non-spherical clusters  
✅ **Noise detection** - Identifies outliers automatically  
✅ **Robust** - Not affected by outliers  
✅ **Intuitive parameters** - eps and min_samples have clear meaning  

### DBSCAN Disadvantages:
❌ **Sensitive to parameters** - Requires careful tuning of eps and min_samples  
❌ **Varying densities** - Struggles with clusters of different densities  
❌ **High dimensions** - Performance degrades in high-dimensional spaces  
❌ **No cluster centers** - Unlike K-Means, doesn't provide centroids  

### Parameter Guidelines:

**eps (ε):**
- Use k-distance graph to find elbow
- Start with average distance to k-nearest neighbors
- Too small: Many small clusters and noise
- Too large: All points in one cluster

**min_samples:**
- Rule of thumb: min_samples ≥ dimensions + 1
- Larger values: More robust to noise, but may miss small clusters
- Smaller values: More sensitive, may create many small clusters
- Typical range: 3-10 for 2D data

### When to Use DBSCAN:
✅ Unknown number of clusters  
✅ Non-spherical cluster shapes  
✅ Presence of noise/outliers  
✅ Geospatial data (hotspot detection)  
✅ Anomaly detection  
✅ Varying cluster sizes  

### When NOT to Use DBSCAN:
❌ Clusters with very different densities  
❌ High-dimensional data (curse of dimensionality)  
❌ Need cluster centers/centroids  
❌ All points must be assigned to clusters  

### DBSCAN vs K-Means:

| Feature | DBSCAN | K-Means |
|---------|--------|----------|
| **Number of clusters** | Automatic | Must specify K |
| **Cluster shape** | Arbitrary | Spherical only |
| **Outlier handling** | Detects noise | Assigns all points |
| **Scalability** | O(n log n) with index | O(n·k·i) |
| **Parameters** | eps, min_samples | K, max_iter |
| **Deterministic** | Yes | No (random init) |

### Real-World Applications:
1. **Geospatial Analysis** - Crime hotspots, disease outbreaks
2. **Customer Segmentation** - Identify customer groups and outliers
3. **Anomaly Detection** - Fraud detection, network intrusion
4. **Image Segmentation** - Object detection in images
5. **Astronomy** - Galaxy cluster detection
6. **Biology** - Gene expression analysis

### Best Practices:
1. **Scale your data** - DBSCAN is sensitive to feature scales
2. **Use k-distance graph** - To find optimal eps
3. **Try different parameters** - Grid search over eps and min_samples
4. **Validate results** - Use silhouette score, domain knowledge
5. **Investigate noise** - Outliers may contain valuable insights
6. **Consider HDBSCAN** - For varying density clusters

### Next Steps:
1. Try HDBSCAN (Hierarchical DBSCAN) for varying densities
2. Combine with PCA for high-dimensional data
3. Use for anomaly detection in your domain
4. Apply to geospatial problems
5. Compare with other clustering algorithms